In [ ]:
import pandas as pd
import numpy as np


In [225]:
df = pd.read_csv('data/test_Batting_Card.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101574 entries, 0 to 101573
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Match ID    101574 non-null  uint32 
 1   innings     101574 non-null  uint8  
 2   team        101574 non-null  object 
 3   batsman     101574 non-null  uint32 
 4   runs        90205 non-null   float64
 5   balls       68343 non-null   float64
 6   fours       80264 non-null   float64
 7   sixes       90167 non-null   float64
 8   strikeRate  90205 non-null   float64
 9   isOut       101574 non-null  bool   
 10  wicketType  101574 non-null  object 
 11  fielders    60540 non-null   object 
 12  bowler      31509 non-null   float64
dtypes: bool(1), float64(6), object(3), uint32(2), uint8(1)
memory usage: 7.9+ MB


In [223]:
from dataclasses import dataclass
@dataclass
class DataTypeInfo:
    IsNumeric = False
    MaxValue = None
    CurrentDataType = None
    OptimizedDataType = None
    CanBeInt = False

def __get_best_int(dataTypeInfo : DataTypeInfo):
    if dataTypeInfo.MaxValue < 256:
        return np.uint8.__name__
    elif 255 < dataTypeInfo.MaxValue < 65535:
        return np.uint16.__name__
    elif 65535 < dataTypeInfo.MaxValue < 4294967295:
        return np.uint32.__name__
    else:
        return np.uint64.__name__

def get_optimized_data_types(df : pd.DataFrame):
    retval : dict = dict()
    for i, col in enumerate(df.columns,1):
        dataTypeInfo : DataTypeInfo = DataTypeInfo()
        dataTypeInfo.CurrentDataType = df[col].dtype
        if pd.api.types.is_integer_dtype(df[col]) or pd.api.types.is_float_dtype(df[col]):
            dataTypeInfo.IsNumeric = True
            dataTypeInfo.MaxValue = max(df[col])
            if pd.api.types.is_integer_dtype(df[col]):
                dataTypeInfo.OptimizedDataType = __get_best_int(dataTypeInfo)
                # print(f'Max Value: {max_val}; Current data type: {current_dtype}; Possile optimized Data type: {optimized_dtype}')
            elif pd.api.types.is_float_dtype(df[col]):
                df1 = df[df[col].isna() == False]
                if not any([r for r in df[df[col].isna() == False].itertuples() if r[i] != int(r[i])]):
                    dataTypeInfo.CanBeInt = True
                    dataTypeInfo.OptimizedDataType = __get_best_int(dataTypeInfo)
        else:
            dataTypeInfo.IsNumeric = False
        retval[col] = dataTypeInfo
    col : str = None
    dataTypeInfo : DataTypeInfo = None
    log = open('log.csv','w')
    for col, dataTypeInfo in retval.items():
        if dataTypeInfo.IsNumeric:
            log.write(f'{col},{dataTypeInfo.MaxValue},{dataTypeInfo.CurrentDataType},{dataTypeInfo.OptimizedDataType}\n')
    log.close()
    


In [232]:
get_optimized_data_types(df)
print('DONE')

DONE


In [247]:
from csv import reader
def optimize_df_data_types(df):
    with open('log.csv') as f:
        for row in reader(f):
            (col,_,_,new_dtype) = row
            try:
                if new_dtype == 'uint8':
                    df[col] = df[col].astype(np.uint8)
                elif new_dtype == 'uint16':
                    df[col] = df[col].astype(np.uint16)
                elif new_dtype == 'uint32':
                    df[col] = df[col].astype(np.uint32)
                print(f'Converted successfully col: {col}')
            except Exception as ex:
                print(f'Couldnt conver col: {col}')
                print(ex)
        print("Done")
 

In [248]:
optimize_df_data_types(df)

Converted successfully col: Match ID
Converted successfully col: innings
Converted successfully col: batsman
Couldnt conver col: runs
Cannot convert non-finite values (NA or inf) to integer
Couldnt conver col: balls
Cannot convert non-finite values (NA or inf) to integer
Couldnt conver col: fours
Cannot convert non-finite values (NA or inf) to integer
Couldnt conver col: sixes
Cannot convert non-finite values (NA or inf) to integer
Converted successfully col: strikeRate
Couldnt conver col: bowler
Cannot convert non-finite values (NA or inf) to integer
Done


In [261]:
df[(df['balls'].isna()) & (df['wicketType']!='DNB')]

,Match ID,innings,team,batsman,runs,balls,fours,sixes,strikeRate,isOut,wicketType,fielders,bowler
375,63350,1,West Indies,1553,6.0,NaN,NaN,0.0,0.0,True,caught,['1428'],NaN
382,63350,1,West Indies,1435,24.0,NaN,5.0,0.0,0.0,True,caught,['1490'],NaN
388,63350,1,West Indies,1497,21.0,NaN,NaN,0.0,0.0,True,caught,['1438'],NaN
394,63350,1,West Indies,1286,23.0,NaN,NaN,0.0,0.0,True,caught,['1490'],NaN
401,63350,1,West Indies,1697,0.0,NaN,0.0,0.0,0.0,True,lbw,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100528,62451,3,England,170,2.0,NaN,0.0,0.0,0.0,True,caught,['140'],NaN
100529,62451,4,Australia,178,64.0,NaN,5.0,0.0,0.0,False,not out,['-'],NaN
100530,62451,4,Australia,172,29.0,NaN,4.0,0.0,0.0,True,caught,['215'],NaN
100531,62451,4,Australia,210,0.0,NaN,0.0,0.0,0.0,True,lbw,NaN,NaN
